In [1]:
import vcf

- indels are detected in poly A and poly T sites, or in the TA repeats. 

These issue is addressed here : http://seqanswers.com/forums/showthread.php?t=47721
- so, we don't take them into account

In [4]:
vcf_reader = vcf.Reader(open('./samtools_out/lsd_2_vars.vcf', 'r'))
counter = 0
for record in vcf_reader:
    if "INDEL" in record.INFO.keys():
        print(record)
        print(record.INFO["DP"])
        print(record.INFO["DP4"])
        print(record.INFO)
        print("--------------------")
        counter += 1

print(counter)
    

Record(CHROM=MH893760.2, POS=11566, REF=TTTATTAT, ALT=[TTTAT])
23
[1, 1, 3, 8]
{'INDEL': True, 'IDV': 8, 'IMF': 0.347826, 'DP': 23, 'VDB': 0.579086, 'SGB': -0.676189, 'MQSB': 0.785785, 'MQ0F': 0.173913, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [1, 1, 3, 8], 'MQ': 13, 'FQ': -999.0, 'PV4': [1.0, 1.0, 1.0, 1.0]}
--------------------
Record(CHROM=MH893760.2, POS=12057, REF=GTTTTTTTTTTT, ALT=[GTTTTTTTTTTTTT])
37
[1, 0, 0, 11]
{'INDEL': True, 'IDV': 4, 'IMF': 0.108108, 'DP': 37, 'VDB': 0.118797, 'SGB': -0.676189, 'MQSB': 1.0, 'MQ0F': 0.0810811, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [1, 0, 0, 11], 'MQ': 21, 'FQ': -999.0, 'PV4': [0.0833333, 1.0, 1.0, 1.0]}
--------------------
Record(CHROM=MH893760.2, POS=16010, REF=TAAAAAAAAA, ALT=[TAAAAAAAAAA])
15
[0, 0, 1, 5]
{'INDEL': True, 'IDV': 4, 'IMF': 0.266667, 'DP': 15, 'VDB': 0.258434, 'SGB': -0.616816, 'MQSB': 1.0, 'MQ0F': 0.0, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [0, 0, 1, 5], 'MQ': 33, 'FQ': -999.0}
--------------------
Record(CHROM=MH893760.2, POS=18168, REF=CTTTTTTT

- we take only SNPs.
- if number of reads supporting alternative variant is greater than 10, we print it out

In [5]:
vcf_reader = vcf.Reader(open('./samtools_out/lsd_2_vars_snp.vcf', 'r'))
counter = 0
for record in vcf_reader:
    if int(record.INFO["DP4"][2]) + (record.INFO["DP4"][3]) >= 10:
        print(record)
        print(record.INFO["DP"])
        print(record.INFO["DP4"])
        print(record.INFO)
        print("--------------------")
        counter += 1
print(counter)
    

Record(CHROM=MH893760.2, POS=21695, REF=C, ALT=[T])
88
[12, 7, 30, 27]
{'DP': 88, 'VDB': 0.00825332, 'SGB': -0.693147, 'RPB': 0.0293038, 'MQB': 5.50095e-08, 'MQSB': 0.962405, 'BQB': 0.0458218, 'MQ0F': 0.0681818, 'AF1': 0.510155, 'AC1': 1.0, 'DP4': [12, 7, 30, 27], 'MQ': 36, 'FQ': -999.0, 'PV4': [0.594997, 1.0, 1.0, 1.0]}
--------------------
Record(CHROM=MH893760.2, POS=24453, REF=T, ALT=[C])
20
[1, 3, 6, 10]
{'DP': 20, 'VDB': 0.0529202, 'SGB': -0.689466, 'RPB': 0.366241, 'MQB': 0.0103432, 'MQSB': 0.962269, 'BQB': 0.982301, 'MQ0F': 0.05, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [1, 3, 6, 10], 'MQ': 38, 'FQ': -999.0, 'PV4': [1.0, 0.233036, 1.0, 1.0]}
--------------------
Record(CHROM=MH893760.2, POS=63853, REF=G, ALT=[A])
77
[7, 3, 8, 55]
{'DP': 77, 'VDB': 0.00144445, 'SGB': -0.693147, 'RPB': 0.876526, 'MQB': 2.84408e-06, 'MQSB': 0.0866522, 'BQB': 0.537055, 'MQ0F': 0.012987, 'AF1': 1.0, 'AC1': 1.0, 'DP4': [7, 3, 8, 55], 'MQ': 37, 'FQ': -999.0, 'PV4': [0.000337164, 1.0, 1.0, 1.0]}
----------------

#### those three SNPs showed in IGV:

![21695](./samtools_out/snp_21695.png)

![snp_24453.png](./samtools_out/snp_24453.png)


![snp_63853.png](./samtools_out/snp_63853.png)

In [8]:
from Bio import SeqIO
from Bio import AlignIO
from Bio import Entrez
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio.SeqUtils import GC
import glob
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
from math import log2
from operator import itemgetter
from random import randint
from time import sleep, time
sns.set()

In [7]:
def blast_fasta(path_to, e_thresh=0.1, hits_to_return=10):
    
    from http.client import IncompleteRead
    from socket import gaierror
    from urllib.error import HTTPError
    """
    takes fasta file and blasts all the record found in it
    writes results into the txt file
    args:
    path_to - path to fasta file
    e_thresh - e-value cut off, 0.1 by default
    hits_to_return - a number of hits returned, 10 by default
    """
    fasta_to_blast = SeqIO.parse(path_to, "fasta")
    

    with open("blast_results_seqman.txt", "w") as f_obj:
        start = time()
        for rec in fasta_to_blast:
            f_obj.write("\nQUERY: " + rec.id + "\n")
            
            try:
                result_handle = NCBIWWW.qblast("blastn", "nt",  rec.seq, hitlist_size=hits_to_return)
                blast_record = NCBIXML.read(result_handle)
                for alignt in  blast_record.alignments:
                    for hsp in alignt.hsps:
                        if hsp.expect < e_thresh:
                            f_obj.write("---------hit--------\n")
                            f_obj.write("sequence: " + alignt.title + "\n")
                            f_obj.write("length: " + str(alignt.length) + "\n")
                            f_obj.write("e value: " + str(hsp.expect) + "\n")
                   
                end = time()
                print(rec.id + " blast query was finished in {0} minutes {1} seconds".format((end - start) // 60, int((end - start) % 60)))
            
            #TODO: make log of the exceptions, to find out what kind of they are
            except IncompleteRead as e: 
                print("Network problem: ", e)
                print("Second and final attempt is under way...")
                
                result_handle = NCBIWWW.qblast("blastn", "nt",  rec.seq, hitlist_size=hits_to_return)
                blast_record = NCBIXML.read(result_handle)
                for alignt in  blast_record.alignments:
                    for hsp in alignt.hsps:
                        if hsp.expect < e_thresh:
                            f_obj.write("---------hit--------\n")
                            f_obj.write("sequence: " + alignt.title + "\n")
                            f_obj.write("length: " + str(alignt.length) + "\n")
                            f_obj.write("e value: " + str(hsp.expect) + "\n")
                        
                   
                end = time()
                print(rec.id + " blast query was finished in {0} minutes {1} seconds".format((end - start) // 60, int((end - start) % 60)))
            
            except gaierror as e:
                print("some other problem, 'gaierror': ", e)
            except HTTPError as e:
                print("urllib.error.HTTPError: ", e)

In [ ]:
blast_fasta(path_to="./input_data/over_seq.txt", hits_to_return=1)

1 blast query was finished in 0.0 minutes 19 seconds
2 blast query was finished in 4.0 minutes 9 seconds
3 blast query was finished in 7.0 minutes 58 seconds
4 blast query was finished in 13.0 minutes 43 seconds
5 blast query was finished in 13.0 minutes 55 seconds
6 blast query was finished in 27.0 minutes 39 seconds
7 blast query was finished in 28.0 minutes 45 seconds
8 blast query was finished in 30.0 minutes 26 seconds
9 blast query was finished in 31.0 minutes 10 seconds
